# RAG Building Blocks

## Useful links

- https://platform.openai.com/settings/proj_QpRaSNGL97yQMNE2Ba1ToHN4/api-keys
- https://python.langchain.com/docs/integrations/document_loaders/
- https://python.langchain.com/docs/how_to/#text-splitters
- https://platform.openai.com/docs/guides/embeddings/embedding-models#embedding-models
- https://python.langchain.com/docs/integrations/vectorstores/chroma/
- https://smith.langchain.com/hub
- https://www.promptingguide.ai/
- https://github.com/promptslab/Awesome-Prompt-Engineering
- https://python.langchain.com/docs/integrations/chat/openai/#model-features
- https://platform.openai.com/docs/quickstart?api-mode=chat

In [ ]:
pip install gradio langchain_community pypdf beautifulsoup4 langchain-text-splitters langchain_openai langchain-chroma

In [ ]:
import os
import re
import gradio
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI

In [ ]:
from google.colab import userdata

In [ ]:
import chromadb.api

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
def predict_old(message, history):
  # Phase 1

  # Step 1 - Document loader
  file_path = "/content/solana-whitepaper-en.pdf"
  loader = PyPDFLoader(file_path, mode='single')

  docs = loader.load()

  # Step 1.5 - Document cleaning
  for doc in docs:
    # doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content = re.sub('\n', '', doc.page_content)

  # print(docs)

  # Step 2- Document Transform / Splitting
  chunk_size = 600
  chunk_overlap= chunk_size * 0.2
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
  )
  chunks = text_splitter.split_documents(docs)

  # print(chunks)

  # Step 3 - Embedding
  embedding_model = OpenAIEmbeddings(
      model="text-embedding-3-small"
  )

  chunk_as_strings = [chunk.page_content for chunk in chunks]
  vectors = embedding_model.embed_documents(chunk_as_strings)

  # print(len(vectors))

  # Step 4 - Vector DB Storage
  chromadb.api.client.SharedSystemClient.clear_system_cache()
  # pinecone or mongo are DB for production
  vector_db = Chroma(
      collection_name='solana-text-embedding-3-small',
      embedding_function=embedding_model,
  )
  # the DB will embed and store the chunks
  vector_db.add_documents(chunks)

  # print(relevant_chunks)

  # Phase 2 - RAG - Generation
  # step 2.1 Similarity Search
  relevant_chunks = vector_db.similarity_search(
      query=message,
      k=3
  )
  # print(relevant_chunks)

  # Step 2.2 create final prompt
  prompt = f"""
    Instructions:

    Answer the user Query based on the provided context, or use the provided context to response logically.
    Do not answer question that are not provided on the provided context.
    If the question is not provided in the provided context, answer with 'I don't know'.
    Use many emojis to answer the question.

    End of Instruction

    User Query:
    {message}
    End of User Query

    Context:
    {relevant_chunks}
    End of Context

  """

  # Step 2.3 Call the LLM
  llm = ChatOpenAI(
      model="gpt-4o-mini",
      temperature=0.7,
  )

  ai_msg = llm.invoke(prompt)

  return ai_msg.content


In [ ]:
vector_db = None
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

In [ ]:
def load_information():
  # Phase 1

  # Step 1 - Document loader
  file_path = "/content/solana-whitepaper-en.pdf"
  loader = PyPDFLoader(file_path, mode='single')

  docs = loader.load()

  # Step 1.5 - Document cleaning
  for doc in docs:
    # doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content = re.sub('\n', '', doc.page_content)

  # print(docs)

  # Step 2- Document Transform / Splitting
  chunk_size = 600
  chunk_overlap= chunk_size * 0.2
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
  )
  chunks = text_splitter.split_documents(docs)

  # print(chunks)

  # Step 3 - Embedding
  # embedding_model = OpenAIEmbeddings(
  #     model="text-embedding-3-small"
  # )

  chunk_as_strings = [chunk.page_content for chunk in chunks]
  vectors = embedding_model.embed_documents(chunk_as_strings)

  # print(len(vectors))

  # Step 4 - Vector DB Storage
  chromadb.api.client.SharedSystemClient.clear_system_cache()
  # pinecone or mongo are DB for production
  vector_db = Chroma(
      collection_name='solana-text-embedding-3-small',
      embedding_function=embedding_model,
      persist_directory='./db'
  )
  # the DB will embed and store the chunks
  vector_db.add_documents(chunks)

In [ ]:
def generation_phase(message, history):

  vector_db = Chroma(
      collection_name='solana-text-embedding-3-small',
      embedding_function=embedding_model,
      persist_directory='./db'
  )

  # Phase 2 - RAG - Generation
  # step 2.1 Similarity Search
  relevant_chunks = vector_db.similarity_search(
      query=message,
      k=3
  )
  # print(relevant_chunks)

  # Step 2.2 create final prompt
  prompt = f"""
    Instructions:

    Answer the user Query based on the provided context, or use the provided context to response logically.
    Do not answer question that are not provided on the provided context.
    If the question is not provided in the provided context, answer with 'I don't know'.
    Use many emojis to answer the question.

    End of Instruction

    User Query:
    {message}
    End of User Query

    Context:
    {relevant_chunks}
    End of Context

  """

  # Step 2.3 Call the LLM
  llm = ChatOpenAI(
      model="gpt-4o-mini",
      temperature=0.7,
  )

  ai_msg = llm.invoke(prompt)

  return ai_msg.content


In [ ]:
load_information()

In [ ]:
gradio.ChatInterface(generation_phase).launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9da3ac4060bcf9a07a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9da3ac4060bcf9a07a.gradio.live


In [ ]:
raise('Stopping')

TypeError: exceptions must derive from BaseException

# Implementing Agents

In [ ]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (

    GenerativeAgent,
    GenerativeAgentMemory,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.3 MB/s eta 0:00:00


In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
USER_NAME = "Eduardo"  # The name you want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

<ipython-input-36-2e1430796da2>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.


## Implementing Your First Generative Agent

In [ ]:
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [ ]:
alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

# Defining the Generative Agent: Alexis
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits="curious, creative writer, world traveler",  # Persistent traits of Alexis
    status="exploring the intersection of technology and storytelling",  # Current status of Alexis
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alexis_memory,
)

<ipython-input-37-0ab8d28a53d7>:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings()


In [ ]:

# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is a disciplined and focused individual who is organized and detail-oriented. She is ambitious and driven, with a strong work ethic and a desire to achieve her goals. She is also independent and self-reliant, able to work effectively on her own.


In [ ]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)



# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is reflective, goal-oriented, enjoys nature, looks forward to new experiences, appreciates art, loves reading, and enjoys trying new things.
